In [260]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import networkx as nx
from collections import defaultdict
import math
from sklearn.preprocessing import MinMaxScaler
%matplotlib notebook
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.palettes import Category20
output_notebook()

Loading BokehJS ...

In [261]:
df=pd.read_csv("MaizeCaseFile.csv")

In [262]:
df

,District,KGISDist 1,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Maize_Production
0,BAGALKOT,Bagalkot,23310,3450.06,840.78,4.45,2879
1,BALLARI,Ballari,9630,"5,269.97",586.45,67.36,21292
2,BELAGAVI,Belagavi,7472,9921.94,413.86,260.15,10450
3,BENGALURU,Bengaluru (Urban),35884,105785.02,853.14,5004.00,271975
4,BENGALURU(R),Bengaluru (Rural),99644,80725.12,1081.95,6124.60,593544
5,BIDAR,Bidar,22190,70194.91,1715.02,51.50,1759
6,CHAMARAJANAGAR,Chamarajanagara,28357,48782.34,1827.19,3538.15,105491
7,CHIKKABALLAPURA,Chikkaballapura,56609,69036.55,1844.32,3814.02,226532
8,CHIKKAMAGALURU,Chikkamagaluru,38148,104566.89,1802.15,1639.85,66868
9,CHITRADURGA,Chitradurga,67863,32999.48,1322.43,2923.79,61353


In [263]:
# df = df.drop(10)

In [264]:
df['RRB_AgricultureLoan'] = df['RRB_AgricultureLoan'].str.replace(',', '').astype(float)

In [265]:
new_entry = {'District': 'Vijayanagara', 
             'KGISDist 1': 'Vijayanagara', 
             'TotalNPK': 9629,
             'RRB_AgricultureLoan': 5269.97,
             'KSCARD_PLDBank_AgricultureLoan': 586.45,
             'SowingSeedsDistributed_Maize': 67.36,
             'Maize_Production': 21291}

new_row = pd.DataFrame([new_entry])

df = df.append(new_row, ignore_index=True)

C:\Users\manis\AppData\Local\Temp\ipykernel_25924\1308792599.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)


In [266]:
df

,District,KGISDist 1,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Maize_Production
0,BAGALKOT,Bagalkot,23310,3450.06,840.78,4.45,2879
1,BALLARI,Ballari,9630,5269.97,586.45,67.36,21292
2,BELAGAVI,Belagavi,7472,9921.94,413.86,260.15,10450
3,BENGALURU,Bengaluru (Urban),35884,105785.02,853.14,5004.00,271975
4,BENGALURU(R),Bengaluru (Rural),99644,80725.12,1081.95,6124.60,593544
5,BIDAR,Bidar,22190,70194.91,1715.02,51.50,1759
6,CHAMARAJANAGAR,Chamarajanagara,28357,48782.34,1827.19,3538.15,105491
7,CHIKKABALLAPURA,Chikkaballapura,56609,69036.55,1844.32,3814.02,226532
8,CHIKKAMAGALURU,Chikkamagaluru,38148,104566.89,1802.15,1639.85,66868
9,CHITRADURGA,Chitradurga,67863,32999.48,1322.43,2923.79,61353


In [267]:
df1=pd.read_csv("KAG 2016-17/Agriculture/Agriculture_KAG_2016_17.csv")

In [268]:
x_col=df.columns[2:-1]
x_col

Index(['TotalNPK', 'RRB_AgricultureLoan', 'KSCARD_PLDBank_AgricultureLoan',
       'SowingSeedsDistributed_Maize'],
      dtype='object')

In [269]:
x=df[x_col]
x.head()

,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize
0,23310,3450.06,840.78,4.45
1,9630,5269.97,586.45,67.36
2,7472,9921.94,413.86,260.15
3,35884,105785.02,853.14,5004.00
4,99644,80725.12,1081.95,6124.60


In [270]:
y=df['Maize_Production']
y.head()

0      2879
1     21292
2     10450
3    271975
4    593544
Name: Maize_Production, dtype: int64

In [271]:
x=sm.add_constant(x)
x.head()

,const,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize
0,1.0,23310,3450.06,840.78,4.45
1,1.0,9630,5269.97,586.45,67.36
2,1.0,7472,9921.94,413.86,260.15
3,1.0,35884,105785.02,853.14,5004.00
4,1.0,99644,80725.12,1081.95,6124.60


In [272]:
model=sm.OLS(y,x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Maize_Production   R-squared:                       0.821
Model:                            OLS   Adj. R-squared:                  0.793
Method:                 Least Squares   F-statistic:                     29.74
Date:                Tue, 19 Mar 2024   Prob (F-statistic):           2.32e-09
Time:                        11:05:57   Log-Likelihood:                -389.20
No. Observations:                  31   AIC:                             788.4
Df Residuals:                      26   BIC:                             795.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const                           1.097e+04   3.52e+04      0.311      0.758   -6.14e+04    8.34e+04
TotalNPK                           0.4046      0.544      0.744      0.464      -0.714       1.523
RRB_AgricultureLoan                0.1313      0.442      0.297      0.769      -0.776       1.039
KSCARD_PLDBank_AgricultureLoan   -17.6583     24.369     -0.725      0.475     -67.749      32.433
SowingSeedsDistributed_Maize      42.4093      5.809      7.300      0.000      30.468      54.351
==============================================================================
Omnibus:                       35.706   Durbin-Watson:                   1.670
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              112.184
Skew:                           2.383   Prob(JB):                     4.36e-25
Kurtosis:                      11.009   Cond. No.                     2.58e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.58e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [273]:
y.mean()

134853.12903225806

In [274]:
# target=10

In [275]:
coef=model.params.values[1:]
# coef

In [276]:
# target=5
# target

In [277]:
x.drop('const',axis=1,inplace=True)

In [278]:
x.head()

,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize
0,23310,3450.06,840.78,4.45
1,9630,5269.97,586.45,67.36
2,7472,9921.94,413.86,260.15
3,35884,105785.02,853.14,5004.00
4,99644,80725.12,1081.95,6124.60


In [279]:
import numpy as np
import pandas as pd
from itertools import product

arr = np.arange(0, 1.1, 0.1)

# Generate all possible combinations of elements in arr
combinations = product(arr, repeat=4)

# Create a DataFrame with the combinations
alpha_df = pd.DataFrame(combinations, columns=['Sensitivity (TotalNPK)', 'Sensitivity (RRB_AgricultureLoan)', 'Sensitivity (KSCARD_PLDBank_AgricultureLoan)', 'Sensitivity (SowingSeedsDistributed_Maize)'])

In [280]:
alpha_df.head()

,Sensitivity (TotalNPK),Sensitivity (RRB_AgricultureLoan),Sensitivity (KSCARD_PLDBank_AgricultureLoan),Sensitivity (SowingSeedsDistributed_Maize)
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.1
2,0.0,0.0,0.0,0.2
3,0.0,0.0,0.0,0.3
4,0.0,0.0,0.0,0.4


In [281]:
import pandas as pd
import numpy as np

x_new_all_comb = pd.DataFrame()
target=150000

def computing_new_x(alpha_x):

    temp_df = (x - np.array(alpha_x) * (y.mean() - target) / coef)
    alpha = pd.DataFrame(np.tile(alpha_x, (len(temp_df), 1)),columns=alpha_x.index)
    temp_df = pd.concat([temp_df, alpha], axis=1)
    return temp_df

for index, row in alpha_df.iterrows():
    x_new_all_comb = pd.concat([x_new_all_comb, computing_new_x(row)], ignore_index=True)

In [282]:
x_new_all_comb

,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Sensitivity (TotalNPK),Sensitivity (RRB_AgricultureLoan),Sensitivity (KSCARD_PLDBank_AgricultureLoan),Sensitivity (SowingSeedsDistributed_Maize)
0,23310.000000,3450.060000,840.780000,4.450000,0.0,0.0,0.0,0.0
1,9630.000000,5269.970000,586.450000,67.360000,0.0,0.0,0.0,0.0
2,7472.000000,9921.940000,413.860000,260.150000,0.0,0.0,0.0,0.0
3,35884.000000,105785.020000,853.140000,5004.000000,0.0,0.0,0.0,0.0
4,99644.000000,80725.120000,1081.950000,6124.600000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
453866,91432.840606,242097.112612,289.695189,522.159272,1.0,1.0,1.0,1.0
453867,128822.840606,154902.862612,-282.344811,357.159272,1.0,1.0,1.0,1.0
453868,154668.840606,238363.282612,-325.274811,371.509272,1.0,1.0,1.0,1.0
453869,110985.840606,173653.552612,622.515189,5649.159272,1.0,1.0,1.0,1.0


# Neg values

In [283]:
def clip(col):
    return np.maximum(col, 0)

x_new_all_comb = x_new_all_comb.apply(clip)
x_new_all_comb

,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Sensitivity (TotalNPK),Sensitivity (RRB_AgricultureLoan),Sensitivity (KSCARD_PLDBank_AgricultureLoan),Sensitivity (SowingSeedsDistributed_Maize)
0,23310.000000,3450.060000,840.780000,4.450000,0.0,0.0,0.0,0.0
1,9630.000000,5269.970000,586.450000,67.360000,0.0,0.0,0.0,0.0
2,7472.000000,9921.940000,413.860000,260.150000,0.0,0.0,0.0,0.0
3,35884.000000,105785.020000,853.140000,5004.000000,0.0,0.0,0.0,0.0
4,99644.000000,80725.120000,1081.950000,6124.600000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
453866,91432.840606,242097.112612,289.695189,522.159272,1.0,1.0,1.0,1.0
453867,128822.840606,154902.862612,0.000000,357.159272,1.0,1.0,1.0,1.0
453868,154668.840606,238363.282612,0.000000,371.509272,1.0,1.0,1.0,1.0
453869,110985.840606,173653.552612,622.515189,5649.159272,1.0,1.0,1.0,1.0


In [284]:
x_new_all_comb=sm.add_constant(x_new_all_comb)
x_new_all_comb

,const,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Sensitivity (TotalNPK),Sensitivity (RRB_AgricultureLoan),Sensitivity (KSCARD_PLDBank_AgricultureLoan),Sensitivity (SowingSeedsDistributed_Maize)
0,1.0,23310.000000,3450.060000,840.780000,4.450000,0.0,0.0,0.0,0.0
1,1.0,9630.000000,5269.970000,586.450000,67.360000,0.0,0.0,0.0,0.0
2,1.0,7472.000000,9921.940000,413.860000,260.150000,0.0,0.0,0.0,0.0
3,1.0,35884.000000,105785.020000,853.140000,5004.000000,0.0,0.0,0.0,0.0
4,1.0,99644.000000,80725.120000,1081.950000,6124.600000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
453866,1.0,91432.840606,242097.112612,289.695189,522.159272,1.0,1.0,1.0,1.0
453867,1.0,128822.840606,154902.862612,0.000000,357.159272,1.0,1.0,1.0,1.0
453868,1.0,154668.840606,238363.282612,0.000000,371.509272,1.0,1.0,1.0,1.0
453869,1.0,110985.840606,173653.552612,622.515189,5649.159272,1.0,1.0,1.0,1.0


In [285]:
y_new_all_comb=sm.add_constant(x_new_all_comb.iloc[:,:5]).dot(model.params.values)
y_new_all_comb

0           6193.091100
1           8055.741212
2          19017.163760
3         236528.210910
4         302520.135175
              ...      
453866     96782.698017
453867     98579.853446
453868    120606.171053
453869    307262.102431
453870     63851.677311
Length: 453871, dtype: float64

In [286]:
y_new_all_comb=np.maximum(y_new_all_comb,0)
y_new_all_comb

0           6193.091100
1           8055.741212
2          19017.163760
3         236528.210910
4         302520.135175
              ...      
453866     96782.698017
453867     98579.853446
453868    120606.171053
453869    307262.102431
453870     63851.677311
Length: 453871, dtype: float64

In [287]:
def init_graph(G,node_adj_frame):
    G.add_nodes_from([i for i in range(len(node_adj_frame))])
    labels = {}
    labels = node_adj_frame.columns
    for i in range(len(node_adj_frame)):
        snode = node_adj_frame[labels[0]][i]-1
        temp = node_adj_frame[labels[2]][i]
        if ',' in str(temp):
            sedge_arr = temp.split(',')
            for j in range(0, len(sedge_arr)):
                k = int(sedge_arr[j])
                G.add_edge(snode, k-1)
        elif np.isnan(temp):
            print("ERROR: Not found in the adjacency excel sheet")
        else:
            G.add_edge(snode, int(temp)-1)
    return

def init_graph_attr(G, AdjFile, values):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["District_Name"].tolist()
    node_list.insert(80, "")
    nodeAttr = {}
    init_graph(G, node_adj_frame)
    
    values = values.fillna(0)
    capability_vector = list(zip(*[values]))
    
    node_attri_dict = dict(zip(node_list[:31], capability_vector))
    node_attri_dict = dict((k, v) for k, v in node_attri_dict.items())

    for i in range(len(node_adj_frame)):
        temp = {}
        temp["capabilityvector"] = node_attri_dict[node_list[i]]
        temp["nodeStress"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp

    nx.set_node_attributes(G, nodeAttr)

def addList(l1,l2):
    for i in range(len(l1)):
        l1[i] = l1[i] + l2[i]
    return l1
def divList(l1,k):
    for i in range(len(l1)):
        l1[i] = l1[i]/k
    return l1

def l2_normalization(l1,l2):
    k = 0
    for i in range(len(l1)):
        k+= (l1[i] - l2[i])**2
    return math.sqrt(k)

def get_node_stress(G,dim):
    stress_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStress"] = l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
        except(KeyError):
            pass
    return stress_dict

def get_node_stability(G,dim):
    satbility_dict = {}
    for n in G.nodes():
        centroid = [0]*dim
        neighList = list(G.neighbors(n))
        for nei in neighList:
            try:
                centroid = addList(centroid,list(G.nodes[nei]["capabilityvector"]))
            except(KeyError):
                pass
        try:
            G.nodes[n]["nodeStability"] = 1 - l2_normalization(divList(centroid,len(neighList)),list(G.nodes[n]["capabilityvector"]))
        except(KeyError):
            pass
        try:
            satbility_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStability"]
        except(KeyError):
            pass
    return satbility_dict

In [288]:
x_new_all_comb['Maize_Production']=y_new_all_comb

In [289]:
x_new_all_comb.drop("const",axis=1,inplace=True)
x_new_all_comb

,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Sensitivity (TotalNPK),Sensitivity (RRB_AgricultureLoan),Sensitivity (KSCARD_PLDBank_AgricultureLoan),Sensitivity (SowingSeedsDistributed_Maize),Maize_Production
0,23310.000000,3450.060000,840.780000,4.450000,0.0,0.0,0.0,0.0,6193.091100
1,9630.000000,5269.970000,586.450000,67.360000,0.0,0.0,0.0,0.0,8055.741212
2,7472.000000,9921.940000,413.860000,260.150000,0.0,0.0,0.0,0.0,19017.163760
3,35884.000000,105785.020000,853.140000,5004.000000,0.0,0.0,0.0,0.0,236528.210910
4,99644.000000,80725.120000,1081.950000,6124.600000,0.0,0.0,0.0,0.0,302520.135175
...,...,...,...,...,...,...,...,...,...
453866,91432.840606,242097.112612,289.695189,522.159272,1.0,1.0,1.0,1.0,96782.698017
453867,128822.840606,154902.862612,0.000000,357.159272,1.0,1.0,1.0,1.0,98579.853446
453868,154668.840606,238363.282612,0.000000,371.509272,1.0,1.0,1.0,1.0,120606.171053
453869,110985.840606,173653.552612,622.515189,5649.159272,1.0,1.0,1.0,1.0,307262.102431


In [290]:
x_new_all_comb['Maize_Production'].mean()

164871.99192619743

In [291]:
dim = 1
G = nx.Graph()
adjacency_file = "Karnataka_District_Adjacency_File - Copy.xlsx"

column = df["Maize_Production"]
column_values = column.values.reshape(-1, 1)
scaler = MinMaxScaler()
normalized_column_values = scaler.fit_transform(column_values)

column_to_pass = pd.Series(normalized_column_values.flatten(), name="Maize_Production")
init_graph_attr(G,adjacency_file , column_to_pass )

initialstability= get_node_stability(G,dim)
df["Initial Stability"] = df["District"].map(initialstability)

In [292]:
df.head()

,District,KGISDist 1,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Maize_Production,Initial Stability
0,BAGALKOT,Bagalkot,23310,3450.06,840.78,4.45,2879,0.957557
1,BALLARI,Ballari,9630,5269.97,586.45,67.36,21292,0.958333
2,BELAGAVI,Belagavi,7472,9921.94,413.86,260.15,10450,0.274673
3,BENGALURU,Bengaluru (Urban),35884,105785.02,853.14,5004.00,271975,0.986036
4,BENGALURU(R),Bengaluru (Rural),99644,80725.12,1081.95,6124.60,593544,0.972711


In [293]:
df.to_excel("MaizeCaseFile_2.xlsx",index=False)

In [294]:
x_new_all_comb[0:31]

,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Sensitivity (TotalNPK),Sensitivity (RRB_AgricultureLoan),Sensitivity (KSCARD_PLDBank_AgricultureLoan),Sensitivity (SowingSeedsDistributed_Maize),Maize_Production
0,23310.0,3450.06,840.78,4.45,0.0,0.0,0.0,0.0,6193.091100
1,9630.0,5269.97,586.45,67.36,0.0,0.0,0.0,0.0,8055.741212
2,7472.0,9921.94,413.86,260.15,0.0,0.0,0.0,0.0,19017.163760
3,35884.0,105785.02,853.14,5004.00,0.0,0.0,0.0,0.0,236528.210910
4,99644.0,80725.12,1081.95,6124.60,0.0,0.0,0.0,0.0,302520.135175
5,22190.0,70194.91,1715.02,51.50,0.0,0.0,0.0,0.0,1062.292292
6,28357.0,48782.34,1827.19,3538.15,0.0,0.0,0.0,0.0,146631.455235
7,56609.0,69036.55,1844.32,3814.02,0.0,0.0,0.0,0.0,172119.734043
8,38148.0,104566.89,1802.15,1639.85,0.0,0.0,0.0,0.0,77855.191126
9,67863.0,32999.48,1322.43,2923.79,0.0,0.0,0.0,0.0,143402.894821


In [295]:
def calculate_and_map_stability(G, df, adjacency_file, column_values, dim):
    G = nx.Graph()
    scaler = MinMaxScaler()
    
    normalized_column_values = scaler.fit_transform(column_values.reshape(-1, 1))
    column_to_pass = pd.Series(normalized_column_values.flatten(), name="Column")

    init_graph_attr(G, adjacency_file, column_to_pass)
    taluka_stress_dict = get_node_stability(G, dim)

    stability_column_name = "New Stability"
    stability_column = df["District"].map(taluka_stress_dict)

    return stability_column, stability_column_name

Cap_Vector = "Maize_Production"

column_values = x_new_all_comb[Cap_Vector].iloc[0:31].values
result_column, result_column_name = calculate_and_map_stability(G, df, adjacency_file, column_values, dim)

df[result_column_name] = result_column

In [296]:
def calculate_and_map_stress(G, df, adjacency_file, column_values, dim):
    G = nx.Graph()
    scaler = MinMaxScaler()
    
    normalized_column_values = scaler.fit_transform(column_values.reshape(-1, 1))
    column_to_pass = pd.Series(normalized_column_values.flatten(), name="Column")

    init_graph_attr(G, adjacency_file, column_to_pass)
    taluka_stress_dict = get_node_stress(G, dim)

    stress_column_name = "New Stress"
    stress_column = df["District"].map(taluka_stress_dict)

    return stress_column, stress_column_name

Cap_Vector = "Maize_Production"

column_values = x_new_all_comb[Cap_Vector].iloc[0:31].values 
result_column, result_column_name = calculate_and_map_stress(G, df, adjacency_file, column_values, dim)

df[result_column_name] = result_column

In [297]:
def calculate_and_map_stability(G, df, adjacency_file, column_values, dim):
    G = nx.Graph()

    scaler = MinMaxScaler()
    normalized_column_values = scaler.fit_transform(column_values.reshape(-1, 1))

    column_to_pass = pd.Series(normalized_column_values.flatten(), name="Column")

    init_graph_attr(G, adjacency_file, column_to_pass)

    stability_dict = get_node_stability(G, dim)
    stability_column = df["District"].map(stability_dict)

    return stability_column

x_new_all_comb["New Stability"] = np.nan
Cap_Vector = "Maize_Production"

chunk_size = 31
for i in range(0, len(x_new_all_comb), chunk_size):

    chunk_values = x_new_all_comb[Cap_Vector].iloc[i:i + chunk_size].values

    result_column = calculate_and_map_stability(G, df, adjacency_file, chunk_values, dim)
    
    x_new_all_comb["New Stability"].iloc[i:i + chunk_size] = result_column

x_new_all_comb["New Stability"] = pd.to_numeric(x_new_all_comb["New Stability"])

In [298]:
def calculate_and_map_stress(G, df, adjacency_file, column_values, dim):
    G = nx.Graph()

    scaler = MinMaxScaler()
    normalized_column_values = scaler.fit_transform(column_values.reshape(-1, 1))

    column_to_pass = pd.Series(normalized_column_values.flatten(), name="Column")

    init_graph_attr(G, adjacency_file, column_to_pass)

    stress_dict = get_node_stress(G, dim)
    stress_column = df["District"].map(stress_dict)

    return stress_column

x_new_all_comb["New Stress"] = np.nan
Cap_Vector = "Maize_Production"

chunk_size = 31
for i in range(0, len(x_new_all_comb), chunk_size):

    chunk_values = x_new_all_comb[Cap_Vector].iloc[i:i + chunk_size].values

    result_column = calculate_and_map_stress(G, df, adjacency_file, chunk_values, dim)
    
    x_new_all_comb["New Stress"].iloc[i:i + chunk_size] = result_column

x_new_all_comb["New Stress"] = pd.to_numeric(x_new_all_comb["New Stress"])

In [299]:
x_new_all_comb

,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Sensitivity (TotalNPK),Sensitivity (RRB_AgricultureLoan),Sensitivity (KSCARD_PLDBank_AgricultureLoan),Sensitivity (SowingSeedsDistributed_Maize),Maize_Production,New Stability,New Stress
0,23310.000000,3450.060000,840.780000,4.450000,0.0,0.0,0.0,0.0,6193.091100,0.958072,0.041928
1,9630.000000,5269.970000,586.450000,67.360000,0.0,0.0,0.0,0.0,8055.741212,0.850815,0.149185
2,7472.000000,9921.940000,413.860000,260.150000,0.0,0.0,0.0,0.0,19017.163760,0.245849,0.754151
3,35884.000000,105785.020000,853.140000,5004.000000,0.0,0.0,0.0,0.0,236528.210910,0.994298,0.005702
4,99644.000000,80725.120000,1081.950000,6124.600000,0.0,0.0,0.0,0.0,302520.135175,0.924640,0.075360
...,...,...,...,...,...,...,...,...,...,...,...
453866,91432.840606,242097.112612,289.695189,522.159272,1.0,1.0,1.0,1.0,96782.698017,0.845867,0.154133
453867,128822.840606,154902.862612,0.000000,357.159272,1.0,1.0,1.0,1.0,98579.853446,0.604688,0.395312
453868,154668.840606,238363.282612,0.000000,371.509272,1.0,1.0,1.0,1.0,120606.171053,0.870850,0.129150
453869,110985.840606,173653.552612,622.515189,5649.159272,1.0,1.0,1.0,1.0,307262.102431,0.942055,0.057945


In [300]:
# x_new_all_comb.to_excel("x_col_new_all_comb_10.xlsx", index =False)

In [301]:
# x_new_all_comb = pd.read_excel('x_col_new_all_comb_10.xlsx')
# x_new_all_comb = x_new_all_comb.dropna()
# x_new_all_comb

In [302]:
# x_new_all_comb.to_excel("x_col_new_all_comb_10.xlsx", index =False)

In [303]:
district_column = pd.DataFrame({
    "KGISDist 1": df["KGISDist 1"].tolist() * (len(x_new_all_comb) // len(df)),
    "District": df["District"].tolist() * (len(x_new_all_comb) // len(df)),
    "Target Maize Production": [target] * len(x_new_all_comb)
})

district_column = pd.concat([district_column, district_column.head(len(x_new_all_comb) % len(df))])

district_column.reset_index(drop=True, inplace=True)

x_new_all_comb["KGISDist 1"] = district_column["KGISDist 1"]
x_new_all_comb["District"] = district_column["District"]
x_new_all_comb["Target Maize Production"] = district_column["Target Maize Production"]

x_new_all_comb.drop(columns=["KGISDist 1", "District", "Target Maize Production"], inplace=True)


x_new_all_comb.insert(0, "District", district_column["District"])
x_new_all_comb.insert(1, "KGISDist 1", district_column["KGISDist 1"])
x_new_all_comb.insert(11, "Target Maize Production", district_column["Target Maize Production"])

In [304]:
x_new_all_comb

,District,KGISDist 1,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Sensitivity (TotalNPK),Sensitivity (RRB_AgricultureLoan),Sensitivity (KSCARD_PLDBank_AgricultureLoan),Sensitivity (SowingSeedsDistributed_Maize),Maize_Production,Target Maize Production,New Stability,New Stress
0,BAGALKOT,Bagalkot,23310.000000,3450.060000,840.780000,4.450000,0.0,0.0,0.0,0.0,6193.091100,150000,0.958072,0.041928
1,BALLARI,Ballari,9630.000000,5269.970000,586.450000,67.360000,0.0,0.0,0.0,0.0,8055.741212,150000,0.850815,0.149185
2,BELAGAVI,Belagavi,7472.000000,9921.940000,413.860000,260.150000,0.0,0.0,0.0,0.0,19017.163760,150000,0.245849,0.754151
3,BENGALURU,Bengaluru (Urban),35884.000000,105785.020000,853.140000,5004.000000,0.0,0.0,0.0,0.0,236528.210910,150000,0.994298,0.005702
4,BENGALURU(R),Bengaluru (Rural),99644.000000,80725.120000,1081.950000,6124.600000,0.0,0.0,0.0,0.0,302520.135175,150000,0.924640,0.075360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453866,UDUPI,Udupi,91432.840606,242097.112612,289.695189,522.159272,1.0,1.0,1.0,1.0,96782.698017,150000,0.845867,0.154133
453867,UTTARA KANNADA,Uttara Kannada,128822.840606,154902.862612,0.000000,357.159272,1.0,1.0,1.0,1.0,98579.853446,150000,0.604688,0.395312
453868,VIJAYAPURA,Vijayapura,154668.840606,238363.282612,0.000000,371.509272,1.0,1.0,1.0,1.0,120606.171053,150000,0.870850,0.129150
453869,YADGIRI,Yadgir,110985.840606,173653.552612,622.515189,5649.159272,1.0,1.0,1.0,1.0,307262.102431,150000,0.942055,0.057945


In [305]:
# x_new_all_comb.to_csv("Maize_Prod_150000.csv", index =False)

In [306]:
# x_new_all_comb.to_excel("Maize_Prod_150000.xlsx", index =False)

In [307]:
# x_new_all_comb = pd.read_csv('Maize_Prod_150000.csv')
df = x_new_all_comb

In [308]:
type(y)

pandas.core.series.Series

In [309]:
y

0       2879
1      21292
2      10450
3     271975
4     593544
5       1759
6     105491
7     226532
8      66868
9      61353
10         0
11       109
12    333937
13     17921
14     19387
15    137154
16    147886
17    576758
18    226327
19    251699
20     90634
21    138476
22    413487
23     24088
24    208090
25      2791
26     14621
27      2673
28       723
29    190252
30     21291
Name: Maize_Production, dtype: int64

In [310]:
df

,District,KGISDist 1,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Sensitivity (TotalNPK),Sensitivity (RRB_AgricultureLoan),Sensitivity (KSCARD_PLDBank_AgricultureLoan),Sensitivity (SowingSeedsDistributed_Maize),Maize_Production,Target Maize Production,New Stability,New Stress
0,BAGALKOT,Bagalkot,23310.000000,3450.060000,840.780000,4.450000,0.0,0.0,0.0,0.0,6193.091100,150000,0.958072,0.041928
1,BALLARI,Ballari,9630.000000,5269.970000,586.450000,67.360000,0.0,0.0,0.0,0.0,8055.741212,150000,0.850815,0.149185
2,BELAGAVI,Belagavi,7472.000000,9921.940000,413.860000,260.150000,0.0,0.0,0.0,0.0,19017.163760,150000,0.245849,0.754151
3,BENGALURU,Bengaluru (Urban),35884.000000,105785.020000,853.140000,5004.000000,0.0,0.0,0.0,0.0,236528.210910,150000,0.994298,0.005702
4,BENGALURU(R),Bengaluru (Rural),99644.000000,80725.120000,1081.950000,6124.600000,0.0,0.0,0.0,0.0,302520.135175,150000,0.924640,0.075360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453866,UDUPI,Udupi,91432.840606,242097.112612,289.695189,522.159272,1.0,1.0,1.0,1.0,96782.698017,150000,0.845867,0.154133
453867,UTTARA KANNADA,Uttara Kannada,128822.840606,154902.862612,0.000000,357.159272,1.0,1.0,1.0,1.0,98579.853446,150000,0.604688,0.395312
453868,VIJAYAPURA,Vijayapura,154668.840606,238363.282612,0.000000,371.509272,1.0,1.0,1.0,1.0,120606.171053,150000,0.870850,0.129150
453869,YADGIRI,Yadgir,110985.840606,173653.552612,622.515189,5649.159272,1.0,1.0,1.0,1.0,307262.102431,150000,0.942055,0.057945


In [311]:
df['Percentage Change'] = df['Maize_Production'] - np.tile(y.values, len(df) // 31 + 1)[:len(df)]

for i in range(0, len(df), 31):
    df.loc[i:i+30, 'Percentage Change'] = df.loc[i:i+30, 'Percentage Change'] * 100 /  y.values
    
df['Percentage Change'] = df['Percentage Change'].replace([np.inf, -np.inf], 0)

In [312]:
df.to_csv("Maize_Prod_150000.csv", index =False)
df.to_excel("Maize_Prod_150000.xlsx", index =False)

In [313]:
# x_new_all_comb['Maternal Deaths'].mean()

In [314]:
# import pandas as pd
# import plotly.express as px
# import ipywidgets as widgets
# from ipywidgets import interact
# from IPython.display import clear_output

# # Function to update the plot based on alpha values
# def update_plot(alpha1, alpha2, alpha3, alpha4):
#     # Filter the dataset based on the selected alpha values
#     filtered_df = df[(df['Sensitivity (Health Facility)'] == alpha1) &
#                      (df['Sensitivity (Caesarean)'] == alpha2) &
#                      (df['Sensitivity (Leprosy Female)'] == alpha3) &
#                      (df['Sensitivity (TB Female)'] == alpha4)]

#     fig = px.scatter(filtered_df, x='New Stability', y='Impact', title='Stability vs Impact',
#                      labels={'Impact': 'Impact', 'New Stability': 'Stability'},
#                      text='KGISDist 1', opacity=1,
#                      color='KGISDist 1', color_discrete_sequence=['darkblue']
#                      )   # Include district names in labels

#     # Customize the layout to make text labels visible
#     fig.update_traces(textposition='top center', showlegend=False)
#     fig.update_layout(annotations=[
#         dict(
#             x=0.5,
#             y=-0.25,
#             showarrow=False,
#             text=f'Alpha 1: {alpha1}, Alpha 2: {alpha2}, Alpha 3: {alpha3}, Alpha 4: {alpha4}',
#             xref="paper",
#             yref="paper"
#         )])

#     # Add average lines
#     avg_stability = filtered_df['New Stability'].mean()
#     avg_impact = filtered_df['Impact'].mean()
    
#     fig.add_shape(
#         type='line',
#         x0=avg_stability, y0=0, x1=avg_stability, y1=1,
#         line=dict(color='red', width=2),
#         yref='paper'
#     )

#     fig.add_shape(
#         type='line',
#         x0=0, y0=avg_impact, x1=1, y1=avg_impact,
#         line=dict(color='red', width=2),
#         xref='paper'
#     )

#     # Clear the output cell to remove the old plot
#     clear_output(wait=True)
#     # Display the new plot
#     display(fig)

# # Create sliders for alphas
# alpha1_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 1:')
# alpha2_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 2:')
# alpha3_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 3:')
# alpha4_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 4:')

# # Create interactive plot
# interactive_plot = interact(update_plot, alpha1=alpha1_slider, alpha2=alpha2_slider, alpha3=alpha3_slider, alpha4=alpha4_slider)


In [315]:
# import pandas as pd
# import plotly.express as px
# import ipywidgets as widgets
# from ipywidgets import interact
# from IPython.display import clear_output

# # Function to update the plot based on alpha values
# def update_plot(alpha1, alpha2, alpha3, alpha4):
#     # Filter the dataset based on the selected alpha values
#     filtered_df = df[(df['Sensitivity (Health Facility)'] == alpha1) &
#                      (df['Sensitivity (Caesarean)'] == alpha2) &
#                      (df['Sensitivity (Leprosy Female)'] == alpha3) &
#                      (df['Sensitivity (TB Female)'] == alpha4)]

#     fig = px.scatter(filtered_df, x='New Stability', y='Impact', title='Stability vs Impact',
#                      labels={'Impact': 'Impact', 'New Stability': 'Stability'},
#                      text='KGISDist 1', opacity=1,
#                      color='KGISDist 1', color_discrete_sequence=['darkblue']
#                      )   # Include district names in labels

#     # Customize the layout to make text labels visible
#     fig.update_traces(textposition='top center', showlegend=False)
#     fig.update_layout(annotations=[
#         dict(
#             x=0.5,
#             y=-0.25,
#             showarrow=False,
#             text=f'Alpha 1: {alpha1}, Alpha 2: {alpha2}, Alpha 3: {alpha3}, Alpha 4: {alpha4}',
#             xref="paper",
#             yref="paper"
#         )])

#     # Add average lines
#     avg_stability = filtered_df['New Stability'].mean()
#     avg_impact = filtered_df['Impact'].mean()
    
#     fig.add_shape(
#         type='line',
#         x0=avg_stability, y0=0, x1=avg_stability, y1=1,
#         line=dict(color='red', width=2),
#         yref='paper'
#     )

#     fig.add_shape(
#         type='line',
#         x0=0, y0=avg_impact, x1=1, y1=avg_impact,
#         line=dict(color='red', width=2),
#         xref='paper'
#     )

#     # Clear the output cell to remove the old plot
#     clear_output(wait=True)
#     # Display the new plot
#     display(fig)

# # Create dropdowns for alphas
# alpha1_dropdown = widgets.Dropdown(options=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], value=0.5, description='Alpha 1:')
# alpha2_dropdown = widgets.Dropdown(options=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], value=0.5, description='Alpha 2:')
# alpha3_dropdown = widgets.Dropdown(options=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], value=0.5, description='Alpha 3:')
# alpha4_dropdown = widgets.Dropdown(options=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0], value=0.5, description='Alpha 4:')

# # Create interactive plot
# interactive_plot = interact(update_plot, alpha1=alpha1_dropdown, alpha2=alpha2_dropdown, alpha3=alpha3_dropdown, alpha4=alpha4_dropdown)


In [316]:
abbreviation_mapping = {
    'BENGALURU': 'BLR',
    'BENGALURU(R)': 'BLR(R)',
    'RAMANAGARA': 'RGA',
    'CHITRADURGA': 'CDA',
    'DAVANAGERE': 'DVG',
    'KOLAR': 'KLR',
    'CHIKKABALLAPURA': 'CKA',
    'SHIVAMOGGA': 'SMG',
    'TUMAKURU': 'TKR',
    'CHIKKAMAGALURU': 'CMG',
    'DAKSHINA KANNADA': 'DKA',
    'UDUPI': 'UPI',
    'HASSAN': 'HSN',
    'KODAGU': 'KDG',
    'MANDYA': 'MDY',
    'MYSURU': 'MYS',
    'CHAMARAJANAGAR': 'CNR',
    'BELAGAVI': 'BLG',
    'VIJAYAPURA': 'VJP',
    'BAGALKOT': 'BKT',
    'DHARAWAD': 'DWD',
    'GADAG': 'GDG',
    'HAVERI': 'HVR',
    'UTTARA KANNADA': 'UTK',
    'BALLARI': 'BLL',
    'BIDAR': 'BDR',
    'KALABURAGI': 'KLB',
    'YADGIRI': 'YDR',
    'RAICHUR': 'RCR',
    'KOPPAL': 'KPL',
}

df['Abbreviation'] = df['District'].map(abbreviation_mapping)

In [317]:
df

,District,KGISDist 1,TotalNPK,RRB_AgricultureLoan,KSCARD_PLDBank_AgricultureLoan,SowingSeedsDistributed_Maize,Sensitivity (TotalNPK),Sensitivity (RRB_AgricultureLoan),Sensitivity (KSCARD_PLDBank_AgricultureLoan),Sensitivity (SowingSeedsDistributed_Maize),Maize_Production,Target Maize Production,New Stability,New Stress,Percentage Change,Abbreviation
0,BAGALKOT,Bagalkot,23310.000000,3450.060000,840.780000,4.450000,0.0,0.0,0.0,0.0,6193.091100,150000,0.958072,0.041928,115.112577,BKT
1,BALLARI,Ballari,9630.000000,5269.970000,586.450000,67.360000,0.0,0.0,0.0,0.0,8055.741212,150000,0.850815,0.149185,-62.165409,BLL
2,BELAGAVI,Belagavi,7472.000000,9921.940000,413.860000,260.150000,0.0,0.0,0.0,0.0,19017.163760,150000,0.245849,0.754151,81.982428,BLG
3,BENGALURU,Bengaluru (Urban),35884.000000,105785.020000,853.140000,5004.000000,0.0,0.0,0.0,0.0,236528.210910,150000,0.994298,0.005702,-13.033106,BLR
4,BENGALURU(R),Bengaluru (Rural),99644.000000,80725.120000,1081.950000,6124.600000,0.0,0.0,0.0,0.0,302520.135175,150000,0.924640,0.075360,-49.031557,BLR(R)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453866,UDUPI,Udupi,91432.840606,242097.112612,289.695189,522.159272,1.0,1.0,1.0,1.0,96782.698017,150000,0.845867,0.154133,561.943082,UPI
453867,UTTARA KANNADA,Uttara Kannada,128822.840606,154902.862612,0.000000,357.159272,1.0,1.0,1.0,1.0,98579.853446,150000,0.604688,0.395312,3587.985539,UTK
453868,VIJAYAPURA,Vijayapura,154668.840606,238363.282612,0.000000,371.509272,1.0,1.0,1.0,1.0,120606.171053,150000,0.870850,0.129150,16581.351460,VJP
453869,YADGIRI,Yadgir,110985.840606,173653.552612,622.515189,5649.159272,1.0,1.0,1.0,1.0,307262.102431,150000,0.942055,0.057945,61.502692,YDR


In [318]:
legend_labels = [{'Abbreviation': abb, 'Full Form': full_form} 
                    for abb, full_form in zip(df['Abbreviation'].unique(), df['KGISDist 1'].unique())]

In [319]:
legend_labels

[{'Abbreviation': 'BKT', 'Full Form': 'Bagalkot'},
 {'Abbreviation': 'BLL', 'Full Form': 'Ballari'},
 {'Abbreviation': 'BLG', 'Full Form': 'Belagavi'},
 {'Abbreviation': 'BLR', 'Full Form': 'Bengaluru (Urban)'},
 {'Abbreviation': 'BLR(R)', 'Full Form': 'Bengaluru (Rural)'},
 {'Abbreviation': 'BDR', 'Full Form': 'Bidar'},
 {'Abbreviation': 'CNR', 'Full Form': 'Chamarajanagara'},
 {'Abbreviation': 'CKA', 'Full Form': 'Chikkaballapura'},
 {'Abbreviation': 'CMG', 'Full Form': 'Chikkamagaluru'},
 {'Abbreviation': 'CDA', 'Full Form': 'Chitradurga'},
 {'Abbreviation': 'DKA', 'Full Form': 'Dakshina Kannada'},
 {'Abbreviation': 'DVG', 'Full Form': 'Davanagere'},
 {'Abbreviation': 'DWD', 'Full Form': 'Dharwad'},
 {'Abbreviation': 'GDG', 'Full Form': 'Gadag'},
 {'Abbreviation': 'HSN', 'Full Form': 'Hassan'},
 {'Abbreviation': 'HVR', 'Full Form': 'Haveri'},
 {'Abbreviation': 'KLB', 'Full Form': 'Kalburgi'},
 {'Abbreviation': 'KDG', 'Full Form': 'Kodagu'},
 {'Abbreviation': 'KLR', 'Full Form': 'Kol

In [320]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import clear_output
import plotly.graph_objects as go

def update_plot(alpha1, alpha2, alpha3, alpha4):
    
    filtered_df = df[(df['Sensitivity (TotalNPK)'] == alpha1) &
                     (df['Sensitivity (RRB_AgricultureLoan)'] == alpha2) &
                     (df['Sensitivity (KSCARD_PLDBank_AgricultureLoan)'] == alpha3) &
                     (df['Sensitivity (SowingSeedsDistributed_Maize)'] == alpha4)]

    fig = px.scatter(filtered_df, x= 'New Stability', y='Percentage Change', title='Percentage Change vs Stability',
                     labels={'Percentage Change': 'Percentage Change', 'New Stability': 'Stability'},
                     text='Abbreviation', opacity=1,color_discrete_sequence=['darkblue'])

    fig.update_traces(textposition='top center', textfont=dict(size=11))

    fig.update_layout(
        annotations=[
            dict(
                x=0.5,
                y=-0.25,
                showarrow=False,
                text=f'α<sub>NPK</sub>: {alpha1:.2f}, α<sub>RRB</sub>: {alpha2:.2f}, α<sub>KS</sub>: {alpha3:.2f}, α<sub>MSD</sub>: {alpha4:.2f}',
                xref="paper",
                yref="paper",
                font=dict(size=13)
            )
        ],
        legend_title_text='Districts',
        legend_title=dict(font=dict(size=12)),
    )

    legend_labels = [{'Abbreviation': abb, 'Full Form': full_form} 
                     for abb, full_form in zip(filtered_df['Abbreviation'].unique(), filtered_df['KGISDist 1'].unique())]

    for label in legend_labels:
        fig.add_trace(go.Scatter(
            x=[None],
            y=[None],
            mode='markers',
            marker=dict(color='darkblue'),
            name=f"{label['Abbreviation']}: {label['Full Form']}"
        ))

    avg_stability = filtered_df['New Stability'].mean()
    avg_precent = filtered_df['Percentage Change'].mean()

    fig.add_shape(
        type='line',
        x0=avg_stability, y0=0, x1=avg_stability, y1=1,
        line=dict(color='red', width=1.5, dash='dash'),
        yref='paper'
    )

    fig.add_shape(
        type='line',
        x0=0, y0=avg_precent, x1=1, y1=avg_precent,
        line=dict(color='red', width=1.5, dash='dash'),
        xref='paper'
    )

    fig.add_trace(go.Scatter(
        x=[avg_stability],
        y=[-0.07],
        text=[f'Avg Stability: {avg_stability:.2f}'],
        mode="text",
        showlegend=False,
        textfont=dict(size=10.7)
    ))

    fig.add_trace(go.Scatter(
        x=[0.3],
        y=[avg_precent-0.04],
        text=[f'Avg Percentage Change: {avg_precent:.2f}'],
        mode="text",
        showlegend=False,
        textfont=dict(size=10.7)
    ))

    clear_output()

    display(fig)
    
alpha1_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αNPK:')
alpha2_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αRRB:')
alpha3_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αKS:')
alpha4_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='αMSD:')

interactive_plot = interact(update_plot, alpha1=alpha1_slider, alpha2=alpha2_slider, alpha3=alpha3_slider, alpha4=alpha4_slider, __manual=True)
update_button = widgets.Button(description="Update Plot")
update_button.on_click(lambda _: update_plot(alpha1_slider.value, alpha2_slider.value, alpha3_slider.value, alpha4_slider.value))
widgets.HBox([update_button, alpha1_slider, alpha2_slider, alpha3_slider, alpha4_slider])


interactive(children=(FloatSlider(value=0.5, description='αNPK:', max=1.0), FloatSlider(value=0.5, description…

In [321]:
# import pandas as pd
# import plotly.express as px
# import ipywidgets as widgets
# from ipywidgets import interact
# from IPython.display import clear_output

# # Function to update the plot based on alpha values
# def update_plot(alpha1, alpha2, alpha3, alpha4):
#     # Filter the dataset based on the selected alpha values
#     filtered_df = df[(df['Sensitivity (Health Facility)'] == alpha1) &
#                      (df['Sensitivity (Caesarean)'] == alpha2) &
#                      (df['Sensitivity (Leprosy Female)'] == alpha3) &
#                      (df['Sensitivity (TB Female)'] == alpha4)]

#     fig = px.scatter(filtered_df, x='New Stability', y='Impact', title='Stability vs Impact',
#                      labels={'Impact': 'Impact', 'New Stability': 'Stability'},
#                      text='KGISDist 1', opacity=1,
#                      color='KGISDist 1', color_discrete_sequence=['darkblue']
#                      )   # Include district names in labels

#     # Customize the layout to make text labels visible
#     fig.update_traces(textposition='top center', showlegend=False)
#     fig.update_layout(annotations=[
#         dict(
#             x=0.5,
#             y=-0.25,
#             showarrow=False,
#             text=f'Alpha 1: {alpha1}, Alpha 2: {alpha2}, Alpha 3: {alpha3}, Alpha 4: {alpha4}',
#             xref="paper",
#             yref="paper"
#         )])

#     # Add average lines
#     avg_stability = filtered_df['New Stability'].mean()
#     avg_impact = filtered_df['Impact'].mean()

#     fig.add_shape(
#         type='line',
#         x0=avg_stability, y0=0, x1=avg_stability, y1=1,
#         line=dict(color='red', width=2),
#         yref='paper'
#     )

#     fig.add_shape(
#         type='line',
#         x0=0, y0=avg_impact, x1=1, y1=avg_impact,
#         line=dict(color='red', width=2),
#         xref='paper'
#     )

#     # Add text annotations
#     fig.add_trace(go.Scatter(
#         x=[avg_stability],
#         y=[0],
#         text=[f'Avg Stability: {avg_stability:.2f}'],
#         mode="text",
#     ))

#     fig.add_trace(go.Scatter(
#         x=[0.1],
#         y=[avg_impact-0.07],
#         text=[f'Avg Impact: {avg_impact:.2f}'],
#         mode="text",
#     ))

#     # Clear the output cell to remove the old plot
#     clear_output()
#     # Display the new plot
#     display(fig)

# # Create sliders for alphas
# alpha1_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 1:')
# alpha2_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 2:')
# alpha3_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 3:')
# alpha4_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.1, description='Alpha 4:')

# # Create button to update plot
# update_button = widgets.Button(description="Update Plot")
# update_button.on_click(lambda _: update_plot(alpha1_slider.value, alpha2_slider.value, alpha3_slider.value, alpha4_slider.value))

# # Organize sliders and button in a vertical box
# controls_box = widgets.VBox([alpha1_slider, alpha2_slider, alpha3_slider, alpha4_slider, update_button])

# # Display the initial plot
# update_plot(alpha1_slider.value, alpha2_slider.value, alpha3_slider.value, alpha4_slider.value)

# # Display the sliders and button below the plot
# widgets.VBox([controls_box])


In [322]:
# import pandas as pd

# def convert_xlsx_to_csv(xlsx_file, csv_file):
#     # Read the Excel file into a pandas DataFrame
#     df = pd.read_excel(xlsx_file)

#     # Save the DataFrame to a CSV file
#     df.to_csv(csv_file, index=False)

# # Replace 'input_file.xlsx' with the path to your Excel file and 'output_file.csv' with the desired CSV file name
# xlsx_file_path = 'x_col_new_all_comb_5.xlsx'
# csv_file_path = 'x_col_new_all_comb_5_c.csv'

# convert_xlsx_to_csv(xlsx_file_path, csv_file_path)